# Calculate Median

In [0]:
%sql
-- Switch to my Catalog
USE CATALOG workspace;

-- Create schema if not exists
CREATE SCHEMA IF NOT EXISTS sql_pyspark_practice;

-- Use this schema
USE sql_pyspark_practice;

In [0]:
create or replace table emp(
emp_id int,
emp_name varchar(20),
department_id int,
salary int,
manager_id int,
emp_age int);

insert into emp
values
(1, 'Ankit', 100,10000, 4, 39),
(2, 'Mohit', 100, 15000, 5, 48),
(3, 'Vikas', 100, 10000,4,37),
(4, 'Rohit', 100, 5000, 2, 16),
(5, 'Mudit', 200, 12000, 6,55),
(6, 'Agam', 200, 12000,2, 14),
(7, 'Sanjay', 200, 9000, 2,13),
(8, 'Ashish', 200,5000,2,12),
(9, 'Mukesh',300,6000,6,51),
(10, 'Rakesh',300,7000,6,50);

Method 1

In [0]:
select * from emp

In [0]:
with cte as (
select *,
        row_number() over(order by emp_age asc) as rn_asc,
        row_number() over(order by emp_age desc) as rn_desc
from emp
-- where emp_id < 10
)

select avg(emp_age) as Median_Age
from cte
where abs(rn_asc - rn_desc) <= 1;

## Method 2
Using Percentile_cont

In [0]:
select  *,
        percentile_cont(0.5) within group (order by emp_age) over() as median
from emp
-- where emp_id < 10

In [0]:
WITH sorted AS (
    SELECT department_id,
           salary,
           ROW_NUMBER() OVER (PARTITION BY department_id ORDER BY salary) AS rn,
           COUNT(*) OVER (PARTITION BY department_id) AS cnt
    FROM emp
)
SELECT department_id,
       AVG(salary) AS median_salary
FROM sorted
WHERE rn IN ((cnt + 1) / 2, (cnt + 2) / 2)
GROUP BY department_id
ORDER BY department_id;
